<a href="https://www.kaggle.com/code/naji468/random-forest?scriptVersionId=98203077" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from category_encoders import MEstimateEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
data_path = "/kaggle/input/house-prices-advanced-regression-techniques/" 
df_train = pd.read_csv(data_path+"train.csv")
df_test = pd.read_csv(data_path+"test.csv")
np.random.seed(386)

In [3]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
features = df_test.columns
X = df_train[features]
Y = df_train.SalePrice
test = df_test[features]


In [5]:

cat_cols = [col for col in features if X[col].dtype == 'object']
encoder = MEstimateEncoder(cols=cat_cols)
X_train = encoder.fit_transform(X, Y)
X_test = encoder.transform(test)
Y_train = Y

In [6]:

my_imputer = SimpleImputer()
X_train = my_imputer.fit_transform(X_train)
X_test = my_imputer.fit_transform(X_test)

In [7]:

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train)

In [8]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

RandomForestRegressor()

In [9]:
val_pred = model.predict(X_val)

In [10]:
rmse = mean_squared_error(np.log(val_pred), np.log(Y_val), squared=False)


In [11]:
print(rmse)

0.1407428401858584


In [12]:
predictions = model.predict(X_test)

In [13]:
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': test.Id,
                       'SalePrice': predictions})
output.to_csv('submission.csv', index=False)
